In [1]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.device_count())  # Should return the number of GPUs
print(torch.cuda.get_device_name(0))  # Should show the GPU model

True
1
NVIDIA RTX 5000 Ada Generation


In [2]:
import transformers
import torch
import os
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM

d:\Anaconda\envs\openfl\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

login("#")
# Don't forget to remove the Key when uploading to GitHub

os.environ["HF_HOME"] = "D:/huggingface_cache" 
os.environ["TRANSFORMERS_CACHE"] = "D:/huggingface_cache"
os.environ["HUGGINGFACE_HUB_CACHE"] = "D:/huggingface_cache"

print("HF_HOME:", os.getenv("HF_HOME"))
print("TRANSFORMERS_CACHE:", os.getenv("TRANSFORMERS_CACHE"))
print("HUGGINGFACE_HUB_CACHE:", os.getenv("HUGGINGFACE_HUB_CACHE"))


transformers.utils.hub.TRANSFORMERS_CACHE = "D:/huggingface_cache"

HF_HOME: D:/huggingface_cache
TRANSFORMERS_CACHE: D:/huggingface_cache
HUGGINGFACE_HUB_CACHE: D:/huggingface_cache


In [4]:
model_name = "mistralai/Mistral-Small-3.1-24B-Base-2503"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="cuda")


device = torch.device("cuda")
model.to(device)

Loading checkpoint shards: 100%|██████████| 10/10 [00:00<00:00, 19.00it/s]
Some weights of MistralForCausalLM were not initialized from the model checkpoint at mistralai/Mistral-Small-3.1-24B-Base-2503 and are newly initialized: ['lm_head.weight', 'model.embed_tokens.weight', 'model.layers.0.input_layernorm.weight', 'model.layers.0.mlp.down_proj.weight', 'model.layers.0.mlp.gate_proj.weight', 'model.layers.0.mlp.up_proj.weight', 'model.layers.0.post_attention_layernorm.weight', 'model.layers.0.self_attn.k_proj.weight', 'model.layers.0.self_attn.o_proj.weight', 'model.layers.0.self_attn.q_proj.weight', 'model.layers.0.self_attn.v_proj.weight', 'model.layers.1.input_layernorm.weight', 'model.layers.1.mlp.down_proj.weight', 'model.layers.1.mlp.gate_proj.weight', 'model.layers.1.mlp.up_proj.weight', 'model.layers.1.post_attention_layernorm.weight', 'model.layers.1.self_attn.k_proj.weight', 'model.layers.1.self_attn.o_proj.weight', 'model.layers.1.self_attn.q_proj.weight', 'model.layers.1.s

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(131072, 5120)
    (layers): ModuleList(
      (0-39): 40 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=5120, out_features=4096, bias=False)
          (k_proj): Linear(in_features=5120, out_features=1024, bias=False)
          (v_proj): Linear(in_features=5120, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=5120, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=5120, out_features=32768, bias=False)
          (up_proj): Linear(in_features=5120, out_features=32768, bias=False)
          (down_proj): Linear(in_features=32768, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((5120,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((5120,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((5120,), eps=1e-

In [14]:
prompt = f"""
        Generate {1} question-answer pairs based on the following text segment. 
        Return the result in valid JSON format as a list of objects.

        Text Segment:
        
Wikipedia is a free online encyclopedia that anyone can edit, and millions already have.

Wikipedia's purpose is to benefit readers by presenting information on all branches of knowledge. Hosted by the Wikimedia Foundation, Wikipedia consists of freely editable content, with articles that usually contain numerous links guiding readers to more information.

Written collaboratively by volunteers known as Wikipedians, Wikipedia articles can be edited by anyone with Internet access, except in limited cases in which editing is restricted to prevent disruption or vandalism. Since its creation on January 15, 2001, it has grown into the world's largest reference website, attracting over a billion visitors each month. Wikipedia currently has more than sixty-four million articles in more than 300 languages, including 6,969,912 articles in English, with 125,967 active contributors in the past month.

Wikipedia's fundamental principles are summarized in its five pillars. While the Wikipedia community has developed many policies and guidelines, new editors do not need to be familiar with them before they start contributing.

Anyone can edit Wikipedia's text, data, references, and images. The quality of content is more important than the expertise of who contributes it. Wikipedia's content must conform with its policies, including being verifiable by published reliable sources. Contributions based on personal opinions, beliefs, or personal experiences, unreviewed research, libellous material, and copyright violations are not allowed, and will not remain. Wikipedia's software makes it easy to reverse errors, and experienced editors watch and patrol bad edits.

Wikipedia differs from printed references in important ways. Anyone can instantly improve it, add quality information, remove misinformation, and fix errors and vandalism. Since Wikipedia is continually updated, encyclopedic articles on major news events appear within minutes. 

        Response Format:
        [
            {{"question": "What is ...?", "answer": "The answer is ..."}},
            {{"question": "How does ... work?", "answer": "It works by ..."}}
        ]

        Question answers should be at least 250 words long.

        Do NOT include any explanation or pre-amble before or after the JSON output.
        Return ONLY valid JSON output.  

        Answer:
        """

In [15]:
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    output_tokens = model.generate(**inputs)

generated_tokens = output_tokens[0][len(inputs["input_ids"][0]):]  
generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

print(generated_text)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [16]:
generated_text

''

In [17]:
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate text
outputs = model.generate(**inputs)

# Decode and print the generated text
generated_text = tokenizer.decode(outputs[0])
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>
        Generate 1 question-answer pairs based on the following text segment. 
        Return the result in valid JSON format as a list of objects.

        Text Segment:
        
Wikipedia is a free online encyclopedia that anyone can edit, and millions already have.

Wikipedia's purpose is to benefit readers by presenting information on all branches of knowledge. Hosted by the Wikimedia Foundation, Wikipedia consists of freely editable content, with articles that usually contain numerous links guiding readers to more information.

Written collaboratively by volunteers known as Wikipedians, Wikipedia articles can be edited by anyone with Internet access, except in limited cases in which editing is restricted to prevent disruption or vandalism. Since its creation on January 15, 2001, it has grown into the world's largest reference website, attracting over a billion visitors each month. Wikipedia currently has more than sixty-four million articles in more than 300 languages, includi

In [18]:
outputs

tensor([[     1,   1010,   1369,  54148,   1032,   1049,   4098,   1045,  24613,
          17684,   4057,   1408,   1278,   3629,   3403,  12158,   1046,   1032,
           1010,   1369,  12162,   1278,   2282,   1294,   5746,  11748,   8174,
           1435,   1261,   2951,   1307,   9536,   1338,   1369,   7269,  88866,
           1877,   1301,   1010,   1087,  36078,   1395,   1261,   5370,   9495,
         108155,   1484,   7855,   1455,  10580,   1710,  11616,   1044,   1321,
          16903,   5314,   1736,   1338,   1087,  36078,   1681,  10485,   1395,
           1317,  14712,  28473,   1536,  34229,   3686,   1408,   1747,  22519,
           1307,   7807,   1046,  32944,   1286,   1536,   1278,  50539,  17364,
           1044,  40377,  15730,   1307,  40444, 104741,   4701,   1044,   1454,
          15837,   1455,   7754,   4231,  15159,  14440,  55907,  28473,   1317,
           2081,   3686,   1338, 109341,  12507,   6655,   1536,  38981,   4629,
           1435, 110136,  10